In [ ]:
## Import Block
from os import path, getcwd
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import json

In [ ]:
## Path to dataset
dataset = path.join(getcwd(), "..", "data", "vgsales_cleaned.csv")

## Read data
df = pd.read_csv(dataset)


In [ ]:
df.describe()
df = df.drop(columns=['Rank', 'Name'], axis=1)
df = df.dropna()
df.head(7)

In [ ]:
# genre = {i: j for i, j in enumerate(df['Genre'].unique().tolist())}
# platform = {i: j for i, j in enumerate(df['Platform'].unique().tolist())}
# publisher = {i: j for i, j in enumerate(df['Publisher'].unique().tolist())}

In [ ]:
# import json

# conf_files = path.join(getcwd(), "config")

# with open(path.join(conf_files, "genre.json"), 'wt', encoding='utf-8') as f:
#     json.dump(genre, f, indent=4)

# with open(path.join(conf_files, "platforms.json"), 'wt', encoding='utf-8') as f:
#     json.dump(platform, f, indent=4)

# with open(path.join(conf_files, "publishers.json"), 'wt', encoding='utf-8') as f:
#     json.dump(publisher, f, indent=4)

In [ ]:
class DataResolver:

    genre = json.loads(
        open(
            path.join(
                getcwd(), 
                "config", 
                "genre.json"
            ), 
            'rt', 
            encoding='utf-8'
        ).read()
    )
    platform = json.loads(
        open(
            path.join(
                getcwd(), 
                "config", 
                "platforms.json"
            ), 
            'rt', 
            encoding='utf-8'
        ).read()
    )
    publisher = json.loads(
        open(
            path.join(
                getcwd(), 
                "config", 
                "publishers.json"
            ), 
            'rt', 
            encoding='utf-8'
        ).read()
    )

    def __init__(self):
        # self.genre = genre
        # self.platform = platform
        # self.publisher = publisher

        pass


    @classmethod
    def resolve_platform(cls, platform):
        for key, value in cls.platform.items():
            if value == platform:
                return key

    @classmethod
    def resolve_genre(cls, genre):
        for key, value in cls.genre.items():
            if value == genre:
                return key

    @classmethod
    def resolve_publisher(cls, publisher):
        for key, value in cls.publisher.items():
            if value == publisher:
                return key


In [ ]:
df['Platform'] = df['Platform'].apply(DataResolver.resolve_platform)
df['Genre'] = df['Genre'].apply(DataResolver.resolve_genre)
df['Publisher'] = df['Publisher'].apply(DataResolver.resolve_publisher)

In [ ]:
# df = df.dropna()
df_dict = df.to_dict('records')

# for item in df_dict:
#     for key in item.keys():
#         item[key] = float(item.get(key, 0))

with open(path.join(getcwd(), "..", "data", "vgsales_ce.json"), "w+t", encoding='utf-8') as dataset_json:
    json.dump(df_dict, dataset_json, indent=4)

# df.to_json(path.join(getcwd(), "..", "data", "vgsales_ce.json"), orient='records')
df.head(25)

In [ ]:
x = df.drop(columns=["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales", "Market_Performance"])
# y = df.drop(columns=["Platform", "Genre", "Publisher"])
# y["NA_Sales"] = y["NA_Sales"].astype(str)
# y["EU_Sales"] = y["EU_Sales"].astype(str)
# y["JP_Sales"] = y["JP_Sales"].astype(str)
# y["Other_Sales"] = y["Other_Sales"].astype(str)
# y["Global_Sales"] = y["Global_Sales"].astype(str)
y = df["Market_Performance"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:

model = DecisionTreeClassifier()
model.fit(x_train, y_train)

In [ ]:
prediction = model.predict(x_test)
accuracy_score(y_test, prediction)

# prediction

In [ ]:
import joblib
from datetime import datetime

joblib.dump(model, "vgsale_model.joblib")



In [ ]:
from sklearn import tree

tree.export_graphviz(
    model,
    out_file="vgsale_tree.dot",
    feature_names=x.columns,
    class_names = sorted(y.unique()),
    label="all",
    rounded=True,
    filled=True
)